# Setup

Edit the file paths and/or options in the first chunk if needed.

Note that to use the REDCap functions, you'll need to create a file in this format:

{
	"Cho Lab Single Cell Sample Metadatabase": {
		"url": "https://redcap.mountsinai.org/redcap/api/",
		"token": "<TOKEN HERE (see API tab on RC)>",
		"content": "project",
		"format": "json",
		"returnFormat": "json"
	}

}


## Imports & Options

In [1]:
import requests
import json
import os
import re
import pandas as pd
import numpy as np
from functions import (get_google_sheet, get_redcap_metadata, 
                       investigate_fields, extract_categories, search_fields)

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_colwidth", 500)

# path_file = None
path_file = "data/sc-metadata-cleanup.csv"

path_config = os.path.join(os.path.expanduser("~"), ".ssh/config_redcap.json")
path_secret = os.path.join(os.path.expanduser("~"),
                           ".ssh/client_secret_google_sheets.json")
rc_project = "Cho Lab Single Cell Sample Metadatabase"
id_sheet = "1PV2vPHjBWxj3Hn0od1B78q98Q6yoyEbj6ku9obzYr2I"
unique_id = "lib_id"
cols_should_be_unique = ["standard_sample_id"]
cols_subject = ["record_id", "record_id1", "grid", "patient_id"]
# cols_subject = ["record_id", "grid"]
fields_cat = ["disease", "disease_status", "organism", "animal_line", 
              "x_chem_version_sc", "inflam_status",
              "tissue_origin", "index_kit",  "instrument"]
missing_ok = []  # TODO: add missing_ok columns

## Data

In [2]:
# Load Google Sheets Database
key_cols = list(pd.unique(cols_should_be_unique + cols_subject))
if path_file is None:
    dff = get_google_sheet(id_sheet, path_secret)
    if any(dff.duplicated(unique_id)):
        raise ValueError(f"{unique_id} repeated; can""t be unique id column.")
    else:
        if unique_id in cols_should_be_unique:
            cols_should_be_unique.remove(unique_id)
else:
    dff = pd.read_csv(path_file)
dff = dff.set_index(unique_id)
print(dff)

# Load REDCap Configuration File
with open(path_config, "r") as json_file:
    config = json.load(json_file)

                record_id         record_id1   organism               project  \
lib_id                                                                          
CD0001          FC_Hu_IL2  Sabic_Project_011      Human              Ileal CD   
CD0002          FC_Hu_IL2  Sabic_Project_011      Human              Ileal CD   
CD0003          FC_Hu_IL3  Sabic_Project_011      Human              Ileal CD   
CD0004          FC_Hu_IL3  Sabic_Project_011      Human              Ileal CD   
HH0001         HH_Hu_MO13  Sabic_Project_020      Human      CRISPR screening   
J00002          JM_Ze_Lck         Sabic_7719  Zebrafish  Zebrafish Felix_Josh   
J00003          JM_Ze_Lck         Sabic_7719  Zebrafish  Zebrafish Felix_Josh   
J00004          JM_Ze_Lck         Sabic_7719  Zebrafish  Zebrafish Felix_Josh   
J00005          JM_Ze_Lck         Sabic_7719  Zebrafish  Zebrafish Felix_Josh   
LC0001          FC_Hu_sCD  Sabic_Project_022      Human                  PBMC   
LC0002          FC_Hu_sCD  S

## REDCap Information

In [3]:
pd.set_option("display.max_colwidth", 50)
drc = get_redcap_metadata("Cho Lab Single Cell Sample Metadatabase", config=config)
print(drc.keys())
data_dict = pd.concat([pd.Series(x, name=x["field_name"]) 
                       for x in drc["data_dictionary"]], axis=1).T
print(data_dict.head())
pd.set_option("display.max_colwidth", 500)

HTTP Status: 200
HTTP Status: 200
dict_keys(['field_names', 'record_ids', 'data_dictionary'])
                  field_name           form_name  \
record_id          record_id  sample_information   
organism            organism  sample_information   
project              project  sample_information   
other_project  other_project  sample_information   
disease              disease  sample_information   

                                                  section_header field_type  \
record_id                                                              text   
organism       <div class="rich-text-field-label"><div class=...      radio   
project                                                            dropdown   
other_project                                                          text   
disease                                                               radio   

                 field_label  \
record_id          Record ID   
organism            Organism   
project              P

### Investigate Fields

Look at category mappings...

In [4]:
for x in fields_cat:
    investigate_fields(x, data_rc=data_dict, data_meta=dff, pattern=True)





disease



field_name                   disease
form_name         sample_information
section_header                      
field_type                     radio
field_label                  Disease
Name: disease, dtype: object

********************************************************************************
Branching Logic:

 [organism] = '1'

********************************************************************************
Categories:

 1, Crohn's Disease | 2, Ulcerative Colitis | 3, Healthy Control | 888, Other, please specify   {other_disease}

********************************************************************************
Unique Metadatabase Values:

 ['CD' nan 'UC' 'Healthy' 'Heathy mice']




disease_status



field_name            disease_status
form_name         sample_information
section_header                      
field_type                     radio
field_label           Disease Status
Name: disease_status, dtype: object

**************************************************

# Database Information

## Repeated Measures

### Improperly Repeated Values

In [5]:
dups = pd.concat([dff.duplicated(subset=x, keep=False) for x in cols_should_be_unique], 
                 keys=cols_should_be_unique, axis=1)  # detect improper duplicates
dups = dups[dups.T.any().T]  # only keep rows where at least 1 improper duplicate
dups = dups.apply(lambda x: x.replace(True, x.name).replace(False, np.nan)).apply(
    lambda y: str(y.dropna().iloc[0]) if any(pd.isnull(y)) else ", ".join(list(y)), 
    axis=1)  # series with text saying which columns duplicated for each row
dff_dup = dff.loc[dups.index].join(dups.to_frame("duplicates"))
print(dff_dup[["duplicates"] + key_cols])

                       duplicates standard_sample_id  record_id  \
lib_id                                                            
LC0001         standard_sample_id        FC_Hu_sCD_1  FC_Hu_sCD   
LC0002         standard_sample_id        FC_Hu_sCD_1  FC_Hu_sCD   
LC0003         standard_sample_id        FC_Hu_sUC_1  FC_Hu_sUC   
LC0004         standard_sample_id        FC_Hu_sUC_1  FC_Hu_sUC   
LC0005         standard_sample_id         FC_Hu_sH_1   FC_Hu_sH   
LC0006         standard_sample_id         FC_Hu_sH_1   FC_Hu_sH   
RL0001         standard_sample_id      RL_Hu_AA2_Inf  RL_Hu_AA2   
RL0001 Repeat  standard_sample_id      RL_Hu_AA2_Inf  RL_Hu_AA2   
RL0002         standard_sample_id      RL_Hu_AA2_Non  RL_Hu_AA2   
RL0002 Repeat  standard_sample_id      RL_Hu_AA2_Non  RL_Hu_AA2   
RL0005         standard_sample_id      RL_Hu_AA3_Inf  RL_Hu_AA3   
RL0006         standard_sample_id      RL_Hu_AA3_Non  RL_Hu_AA3   
RL0007         standard_sample_id      RL_Hu_AA3_Inf  RL_Hu_AA

### Repeated Measures Column Expansion

In [6]:
expansion = dff.groupby("record_id").apply(
    lambda x: list(pd.Series([c if len(pd.unique(x[c])) > 1 else np.nan 
                         for c in dff.drop(cols_should_be_unique, axis=1).columns]).dropna())).apply(
                             lambda y: np.nan if len(y) == 0 else y).dropna()
print(expansion)
cols_to_expand = pd.unique(expansion.explode())
print(cols_to_expand)

record_id
FC_Hu_IL1                                                                                                                                                                                                                                                [inflam_status]
FC_Hu_IL2                                                                                                                                                                         [project_owner_id, inflam_status, no_live_cells, cell_viability_percentage, index_kit]
FC_Hu_IL3                                                                                                                                                                         [project_owner_id, inflam_status, no_live_cells, cell_viability_percentage, index_kit]
FC_Hu_sCD                                                                                                                                                      [record_id1, type_of_experiment, no_

#### Already Expanded REDCap Fields

In [7]:
print(data_dict.loc[data_dict.index.intersection(set(cols_to_expand))])
already_expanded = dict(zip(cols_to_expand, [] * len(cols_to_expand)))
for i in cols_to_expand:
    already_expanded[i] = list(pd.Series([d if i in d and i != d else np.nan for d in data_dict.index]).dropna())
_ = [print(f"{k}: {already_expanded[k]}") for k in already_expanded]

                            field_name           form_name  \
patient_id                  patient_id  sample_information   
project_owner_id      project_owner_id  sample_information   
animal_line                animal_line  sample_information   
sc_process_date        sc_process_date  sample_information   
type_of_experiment  type_of_experiment  sample_information   
x_chem_version_sc    x_chem_version_sc  sample_information   
index_kit                    index_kit        library_prep   
pre_amp_date              pre_amp_date        library_prep   
date_sent                    date_sent          sequencing   
instrument                  instrument          sequencing   

                                                 section_header field_type  \
patient_id                                                            text   
project_owner_id                                                      text   
animal_line                                                           text   
sc_pr

#### (In)Varying Values

In [8]:
unique_val_ct = dff.groupby(cols_subject[0]).apply(
    lambda x: pd.Series([np.nan]) if len(x) == 1 else pd.Series(
        [len(x[c].unique())for c in already_expanded], 
        index=cols_to_expand)).dropna()
invarying_vals = unique_val_ct[unique_val_ct <= 1].dropna().groupby(cols_subject[0]).apply(
    lambda x: ", ".join(list([str(i) for i in x.reset_index(0).index.values])))
varying_vals = unique_val_ct[unique_val_ct > 1].dropna().groupby(cols_subject[0]).apply(
    lambda x: ", ".join(list([str(i) for i in x.reset_index(0).index.values])))
print(f"\n\n{'=' * 80}\n\nInvarying\n\n{'=' * 80}\n{invarying_vals}")
print(f"\n\n{'=' * 80}\n\nVarying\n\n{'=' * 80}\n{varying_vals}")





Invarying

record_id
FC_Hu_IL1     project_owner_id, no_live_cells, cell_viability_percentage, index_kit, record_id1, type_of_experiment, no_live_nuclei, no_nuclei, date_sent, instrument, ref_genome, animal_line, sc_process_date, tissue_origin, pre_amp_date, targ_cell, x_chem_version_sc, repeat_data_release, repeat_seq_platform, patient_id
FC_Hu_IL2                                                                            record_id1, type_of_experiment, no_live_nuclei, no_nuclei, date_sent, instrument, ref_genome, animal_line, sc_process_date, tissue_origin, pre_amp_date, targ_cell, x_chem_version_sc, repeat_data_release, repeat_seq_platform, patient_id
FC_Hu_IL3                                                                            record_id1, type_of_experiment, no_live_nuclei, no_nuclei, date_sent, instrument, ref_genome, animal_line, sc_process_date, tissue_origin, pre_amp_date, targ_cell, x_chem_version_sc, repeat_data_release, repeat_seq_platform, patient_id
FC_Hu_sCD    

# Major Changes

## Initial

In [9]:
# Setup
if any(("_old" in c for c in dff.columns)):
    raise NotImplementedError("Change code!!! '_old' is already in >= 1 original variable")
data = dff.copy()
labels_uninflamed = ["uninflamed", "noninflamed", "non-infl", "non-inflamed", "un-inflamed"]
labels_inflamed = ["inflamed"]

# Project Owner ID
# concatenation of libid_ssids separated by comma, e.g., "CD0001_FC01, CD0002_FC02"
data = data.assign(libbbbbbbb=data.index.values)  # rename old column
data = data.join(data.apply(lambda r: f"{r['project_owner_id']}_{r['libbbbbbbb']}", 
                         axis=1).to_frame("project_owner_id_lib_id")).drop(
                             "libbbbbbbb", axis=1)  # owner_libid
data = data.join(data.groupby(
    cols_subject[0]).apply(lambda x: ",".join(
        x["project_owner_id_lib_id"])).to_frame("project_owner_id"), 
    lsuffix="_old", on=cols_subject[0])[list(data.drop(
        "project_owner_id_lib_id", axis=1).columns) + [
            "project_owner_id_old"]]  # ownder1_libid1,owner2_libid2, etc.

# Disease & Disease Status
data = data.rename({"CD": "Crohn's Disease", "UC": "Ulcerative Colitis", 
                    "Healthy Mice": "Healthy Control", 
                    "Healthy": "Healthy Control"}, axis=1)
data.loc[:, "disease_status_old"] = data.loc[:, "disease_status"].copy()
data.loc[:, "disease_old"] = data.loc[:, "disease"].copy()
cats_disease_status = extract_categories("disease_status", data_dict)
for c in cats_disease_status:
    data.loc[:, "disease_status"] = data["disease_status"].apply(
        lambda x: c if re.sub("In rem", "rem", str(c)).lower() in str(
            x).lower() else x)  # standardize "in remission" variants
# data.loc[:, "disease"] = data.apply(lambda x:  if (pd.isnull(
#     x["disease"])) & (x["organism"] != "Human") else x)  # "other" for fish
data.loc[:, "disease"] = data.disease.apply(lambda x: "Healthy Control" if any(
    ("healthy" in str(x).lower() for i in ["healthy", "hc", "control"])) else x)

# Inflammation Status
data.loc[:, "inflam_status_old"] = data.loc[:, "inflam_status"].copy()
data.loc[:, "inflam_status"] = data["inflam_status"].apply(
    lambda x: np.nan if pd.isnull(x) else str("Non-inflamed" if any(
        (p in str(x).strip("-").lower() for p in labels_uninflamed)) else str(
            "Inflamed" if any(
        (q in str(x).strip("-").lower() for q in labels_inflamed)) else x)))
data.loc[(pd.isnull(data.inflam_status)) & (  # HCs NAs for inflam_status
    data.disease == "Healthy Control"), "inflam_status"] = "Healthy Control"

# Chemistry Version
data.loc[:, "x_chem_version_sc_old"] = data["x_chem_version_sc"].copy()
data.loc[:, "x_chem_version_sc"] = data["x_chem_version_sc"].apply(
    lambda x: "NovaSeq 6000" if "nova" in str(x).lower() else x)
# print(data[["x_chem_version_sc_old", "x_chem_version_sc"]])

# Instrument
data.loc[:, "index_kit_old"] = data["index_kit"].copy()
data.loc[:, "index_kit"] = data["index_kit"].apply(
    lambda x: "Dual Index TT" if "-tt-" in str(x).lower() else "Dual Index TS")
# print(data[["index_kit_old", "index_kit"]])

# Tissue Origin
data.loc[:, "tissue_origin_old"] = data["tissue_origin"].copy()
data = data.rename({"Terminal Ileal Resection": "terminal ileum resection"}, axis=1)
# tissue_fields = search_fields("^tissue_origin", data_dict, 
#                               header=False, print_output=False)  # search fields for partial match
cats_tissue = extract_categories("tissue_origin_1", data_dict)
for c in cats_tissue: 
    data.loc[:, "tissue_origin"] = data["tissue_origin"].apply(
        lambda x: c if c.lower() in str(x).lower() else x)
# print(data[["tissue_origin_old", "tissue_origin"]])

# Print Conversions
print(f"\n\n\n{'=' * 80}\n\nConversions\n\n{'=' * 80}\n\n")
changed_variables = list(np.array(data.columns)[np.where(["_old" in c for c in data.columns])[0]])
for y in [re.sub("_old", "", c) for c in changed_variables]:
    print(f"\n\n*** {y}")
    conv = data.apply(lambda x: str(x[f"{y}_old"]) + " - > " + str(x[y]) if str(x[
        y]).lower() != str(x[f"{y}_old"]).lower() else np.nan, axis=1).dropna().unique()
    print("\n".join(list(conv)))

# Dropped Variables
drop_variables = []
for i in dff.columns:
    fields = search_fields("^" + i, data_dict, header=False, print_output=False)
    if len(fields) == 0:
        drop_variables += [i]
print(f"\n\n\nDROP VARIABLES (not in REDCap):\n\n{drop_variables}")
data = data.drop(drop_variables, axis=1).drop(changed_variables, axis=1)





Conversions





*** project_owner_id
FC01 - > FC01_CD0001,FC02_CD0002
FC02 - > FC01_CD0001,FC02_CD0002
FC03 - > FC03_CD0003,FC04_CD0004
FC04 - > FC03_CD0003,FC04_CD0004
HH_Spin - > HH_Spin_HH0001
JMFC01 - > JMFC01_J00002,JMFC02_J00003,JMFC03_J00004,JMFC04_J00005
JMFC02 - > JMFC01_J00002,JMFC02_J00003,JMFC03_J00004,JMFC04_J00005
JMFC03 - > JMFC01_J00002,JMFC02_J00003,JMFC03_J00004,JMFC04_J00005
JMFC04 - > JMFC01_J00002,JMFC02_J00003,JMFC03_J00004,JMFC04_J00005
Severe CD PBMC - > Severe CD PBMC_LC0001,Severe CD PBMC_LC0002
1 - > 1_LC0003,1_LC0004
2 - > 2_LC0005,2_LC0006
AA4_Inf - > AA4_Inf_RL0001,AA4_Inf_RL0001 Repeat,A4_Non_RL0002,A4_Non_RL0002 Repeat
A4_Non - > AA4_Inf_RL0001,AA4_Inf_RL0001 Repeat,A4_Non_RL0002,A4_Non_RL0002 Repeat
EA2_Inf - > EA2_Inf_RL0003,EA2_Non_RL0004
EA2_Non - > EA2_Inf_RL0003,EA2_Non_RL0004
AA5_Inf - > AA5_Inf_RL0005,AA5_Non_RL0006,AA5_Inf_RL0007,AA5_Non_RL0008,AA5_Inf_RL0009,AA5_Non_RL0010
AA5_Non - > AA5_Inf_RL0005,AA5_Non_RL0006,AA5_Inf_RL0007,AA5_Non_RL

## Manual Changes

### inflam_status

In [10]:
data.loc[(data.standard_sample_id == "SN_Ze_WT3_Non") & (
    pd.isnull(data.inflam_status)), "inflam_status"] = "Untreated"

## Conclusion

In [268]:
investigate_fields("record_id1", data_dict, data, pattern=True)





record_id1



                                                field_name  \
record_id                                        record_id   
organism                                          organism   
project                                            project   
other_project                                other_project   
disease                                            disease   
other_disease                                other_disease   
disease_status                              disease_status   
sample_origin                                sample_origin   
procedure                                        procedure   
grid                                                  grid   
patient_id                                      patient_id   
project_owner_id                          project_owner_id   
animal_line                                    animal_line   
sc_process_date                            sc_process_date   
sc_isolation_prot                        sc_isolatio

In [15]:
# Missingness
missing_bad = {"grid": data.organism == "Human",
               "patient_id": data.organism == "Human",
               "animal_line": data.organism != "Human",
               "disease": data.organism == "Human",
               "disease_status": data.apply(
                   lambda x: (x["organism"] == "Human") and (
                       x["disease"] not in ["Healthy", "Healty mice"]), axis=1)}
missingness = data.apply(lambda x: sum(pd.isnull(x)))
nawah = dict()
for i in missingness[missingness > 0].index.values:
    if i in missing_bad:
        nab = data[pd.isnull(data[i]) & (missing_bad[i])]  # only if NAs unexpected
    else:
        nab = data[pd.isnull(data[i])]  # all NAs
    nawah.update({i: nab})
    print(f"\n\n{i}\n\n--\n\n{nab}\n\n")
na_counts = "\n".join([f"{i}: {nawah[i].shape[0]}" for i in nawah])
print(f"\n\n{'=' * 80}\n\nMissingness\n\n{'=' * 80}\n\n{na_counts}")



disease

--

Empty DataFrame
Columns: [record_id, organism, project, disease, disease_status, grid, patient_id, project_owner_id, animal_line, sc_process_date, type_of_experiment, x_chem_version_sc, standard_sample_id, inflam_status, tissue_origin, no_live_cells, cell_viability_percentage, targ_cell, no_live_nuclei, no_nuclei, index_kit, pre_amp_date, date_sent, instrument]
Index: []




disease_status

--

        record_id organism          project          disease disease_status  \
lib_id                                                                        
CD0001  FC_Hu_IL2    Human         Ileal CD               CD            NaN   
CD0002  FC_Hu_IL2    Human         Ileal CD               CD            NaN   
CD0003  FC_Hu_IL3    Human         Ileal CD               CD            NaN   
CD0004  FC_Hu_IL3    Human         Ileal CD               CD            NaN   
LC0005   FC_Hu_sH    Human             PBMC  Healthy Control            NaN   
LC0006   FC_Hu_sH    Human        

# Data Dictionary

In [17]:
data_dict_new = data_dict.copy()
data_dict.head()

,field_name,form_name,section_header,field_type,field_label,select_choices_or_calculations,field_note,text_validation_type_or_show_slider_number,text_validation_min,text_validation_max,identifier,branching_logic,required_field,custom_alignment,question_number,matrix_group_name,matrix_ranking,field_annotation
record_id,record_id,sample_information,,text,Record ID,,Standardized subject ID (or Sequencing Batch ID if applicable). Please follow the convention: SN_Ze_KO1,,,,,,,,,,,
organism,organism,sample_information,"<div class=""rich-text-field-label""><div class=""rich-text-field-label""> <p style=""text-align: left;"">Clinical/Organism Information</p> </div></div>",radio,Organism,"1, Human | 2, Zebrafish | 3, Mouse",,,,,,,y,,,,,
project,project,sample_information,,dropdown,Project,"1, Ileal CD | 2, UC | 3, Zebrafish | 4, Mouse Organoid | 5, PBMC | 6, Perianal CD | 7, Ileal CD/PTGER4 | 8, PSC-IBD | 888, Other, please specify",What best describes the project you are working on? Only use other if no other choice applies.,,,,,,y,,,,,
other_project,other_project,sample_information,,text,Other Project,,,,,,,[project] = '888',,,,,,
disease,disease,sample_information,,radio,Disease,"1, Crohn's Disease | 2, Ulcerative Colitis | 3, Healthy Control | 888, Other, please specify {other_disease}",,,,,,[organism] = '1',,,,,,


## Construction

### Repeated Measures Transformation

#### Google Data to Wide Format

In [256]:
# Dataframe for Transformation
col_rm = "sample"
data_new = data.groupby(cols_subject[0], group_keys=True).apply(
    lambda x: x.assign(sample=np.arange(1, len(x) + 1))).reset_index(
        data.index.names).set_index(
        "sample", append=True)  # set uniform 1:n "measurement" column for RMs
if cols_subject[0] in data_new.columns:
    data_new = data_new.drop(cols_subject[0], axis=1)
data_new.head()

# Fields Overlapping between Google & REDCap
fields_overlap = [data_dict.index.values[np.where([
    re.search("^" + i + "(_[0-9]+)", x) 
    for x in data_dict.index.values])[0]] for i in data_new.columns]
fields_overlap = pd.Series(fields_overlap, index=pd.Index(
    data_new.columns, name="Google")).explode().dropna().to_frame("REDCap")
# print(fields_overlap)

# Repeated Measures Fields
fields_rm_num = fields_overlap.groupby("Google").apply(
    lambda x: pd.Series(list(x["REDCap"].str.strip(
        f"{x.name}_").astype(int).reset_index(0, drop=True)), 
                        index=pd.Index(x["REDCap"], name="REDCap")).to_frame(
            "Number") if len(x["REDCap"]) > 1 else np.nan)  # measurement #
google_rm_num = data_new.groupby(cols_subject[0]).apply(lambda x: x.apply(
    lambda y: len(y.unique()))).stack()  # Google sheet: # unique values / subject
google_rm_num = google_rm_num[google_rm_num > 1]  # within-subject-varying only
google_rm_num.index.names = [cols_subject[0], "Field"]
cols_needed = google_rm_num.reset_index(0, drop=True).groupby("Field").apply(
    max)  # maximum # unique values / Google column = # REDCap RM categories needed
fields_rm_num.Number.groupby("Google").max().min()
rm_num = fields_rm_num.Number.groupby("Google").max().to_frame("REDCap").join(
    cols_needed.to_frame("Google"))  # # of REDCap RM categories vs. # needed
if any(rm_num.Google > rm_num.REDCap):
    print(rm_num[rm_num.Google < rm_num.REDCap])
    raise ValueError("Code to add more REDCap repeated measures fields required!")
cols_needs_fields = list(set(pd.unique(
    google_rm_num.reset_index(0).index.values)).intersection(
        data_dict.field_name))  # Google RM columns that need fields in RC
print(f"{'=' * 80}\n\n\nNeed new RC RM fields:\n\n{cols_needs_fields}\n\n")

# Long to Wide Format
col_rm_list = cols_needs_fields + list(rm_num.index.values)
if unique_id not in col_rm_list:
    col_rm_list += [unique_id]
data_wide = data_new[col_rm_list].reset_index()
data_wide["column_name"] = data_wide[col_rm].astype(str)
data_wide = data_wide.astype(object).pivot(
    index=cols_subject[0], columns="column_name", values=rm_num.index.values)
data_wide.columns = [f"{c[0]}_{c[1]}" for c in data_wide.columns]
if data_new.drop(col_rm_list, axis=1).groupby(cols_subject[0]).apply(
    lambda s: s.apply(lambda x: len(x.unique()) > 1)).stack().any():
    raise ValueError("Columns varying w/i-subject retained in data_new!")
data_new = data_wide.join(data_new.drop(col_rm_list, axis=1).groupby(
    cols_subject[0]).apply(lambda s: s.apply(
        lambda x: x.unique()[0])))  # join bt- & wi-subject columns (wide)
data_new.to_csv("data/data_new.csv")
data_new.head()




Need new RC RM fields:

['x_chem_version_sc', 'date_sent', 'patient_id', 'pre_amp_date', 'index_kit', 'instrument', 'type_of_experiment', 'animal_line', 'sc_process_date']




,cell_viability_percentage_1,cell_viability_percentage_2,cell_viability_percentage_3,cell_viability_percentage_4,cell_viability_percentage_5,cell_viability_percentage_6,cell_viability_percentage_7,cell_viability_percentage_8,cell_viability_percentage_9,inflam_status_1,inflam_status_2,inflam_status_3,inflam_status_4,inflam_status_5,inflam_status_6,inflam_status_7,inflam_status_8,inflam_status_9,no_live_cells_1,no_live_cells_2,no_live_cells_3,no_live_cells_4,no_live_cells_5,no_live_cells_6,no_live_cells_7,...,standard_sample_id_9,targ_cell_1,targ_cell_2,targ_cell_3,targ_cell_4,targ_cell_5,targ_cell_6,targ_cell_7,targ_cell_8,targ_cell_9,tissue_origin_1,tissue_origin_2,tissue_origin_3,tissue_origin_4,tissue_origin_5,tissue_origin_6,tissue_origin_7,tissue_origin_8,tissue_origin_9,organism,project,disease,disease_status,grid,project_owner_id
record_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FC_Hu_AA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Whole blood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Human,PBMC,Healthy Control,NaN,NaN,nan_QQ0033
FC_Hu_EA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthy Control,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Whole blood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Human,PBMC,Healthy Control,NaN,1010659.0,nan_QQ0034
FC_Hu_IL1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inflamed,Non-inflamed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Terminal Ileal Resection, Biopsies","Terminal Ileal Resection, Biopsies",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Human,Ileal CD/PTGER4,CD,NaN,NaN,"JUCH09 (HIMC ID)_QQ0031,JUCH09 (HIMC ID)_QQ0032"
FC_Hu_IL2,65.2,63.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inflamed,Non-inflamed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,417000.0,100000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,5000,5000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Terminal Ileal Resection, Biopsies","Terminal Ileal Resection, Biopsies",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Human,Ileal CD,CD,NaN,1010694.0,"FC01_CD0001,FC02_CD0002"
FC_Hu_IL3,69.2,72.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inflamed,Non-inflamed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61700.0,296000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,5000,5000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Terminal Ileal Resection, Biopsies","Terminal Ileal Resection, Biopsies",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Human,Ileal CD,CD,NaN,1010695.0,"FC03_CD0003,FC04_CD0004"


#### Data Dictionary

In [257]:
data_dict_new = []
for x in data_dict.index.values:
    if x in cols_needs_fields:
        data_dict_new += [pd.concat([data_dict.loc[x]] * fields_rm_num.Number.max(
            ), keys=[f"{x}_{c}" for c in np.arange(1, fields_rm_num.Number.max())]).unstack(
                1)]  # repeat entry for maximum RMs already anticipated by RC RM fields
    else:
        data_dict_new += [data_dict.loc[[x]]]  # if w/i-subject-invarying, just one RC row
data_dict_new = pd.concat(data_dict_new)
data_dict_new.to_csv("data/data_dictionary_new.csv")
data_dict_new.head()

,field_name,form_name,section_header,field_type,field_label,select_choices_or_calculations,field_note,text_validation_type_or_show_slider_number,text_validation_min,text_validation_max,identifier,branching_logic,required_field,custom_alignment,question_number,matrix_group_name,matrix_ranking,field_annotation
record_id,record_id,sample_information,,text,Record ID,,Standardized subject ID (or Sequencing Batch ID if applicable). Please follow the convention: SN_Ze_KO1,,,,,,,,,,,
organism,organism,sample_information,"<div class=""rich-text-field-label""><div class=""rich-text-field-label""> <p style=""text-align: left;"">Clinical/Organism Information</p> </div></div>",radio,Organism,"1, Human | 2, Zebrafish | 3, Mouse",,,,,,,y,,,,,
project,project,sample_information,,dropdown,Project,"1, Ileal CD | 2, UC | 3, Zebrafish | 4, Mouse Organoid | 5, PBMC | 6, Perianal CD | 7, Ileal CD/PTGER4 | 8, PSC-IBD | 888, Other, please specify",What best describes the project you are working on? Only use other if no other choice applies.,,,,,,y,,,,,
other_project,other_project,sample_information,,text,Other Project,,,,,,,[project] = '888',,,,,,
disease,disease,sample_information,,radio,Disease,"1, Crohn's Disease | 2, Ulcerative Colitis | 3, Healthy Control | 888, Other, please specify {other_disease}",,,,,,[organism] = '1',,,,,,


### New Field Categories

In [265]:
pd.isnull(data_new).any()

cell_viability_percentage_1     True
cell_viability_percentage_2     True
cell_viability_percentage_3     True
cell_viability_percentage_4     True
cell_viability_percentage_5     True
cell_viability_percentage_6     True
cell_viability_percentage_7     True
cell_viability_percentage_8     True
cell_viability_percentage_9     True
inflam_status_1                False
inflam_status_2                 True
inflam_status_3                 True
inflam_status_4                 True
inflam_status_5                 True
inflam_status_6                 True
inflam_status_7                 True
inflam_status_8                 True
inflam_status_9                 True
no_live_cells_1                 True
no_live_cells_2                 True
no_live_cells_3                 True
no_live_cells_4                 True
no_live_cells_5                 True
no_live_cells_6                 True
no_live_cells_7                 True
no_live_cells_8                 True
no_live_cells_9                 True
n